# Малютин Павел, ИТПЗ РАН
 

In [4]:
import pandas as pd
import math 
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from numba import jit, njit
from sklearn.linear_model import LinearRegression
import random

# b-value calculating

In [5]:
'''чтение файла из директории проги
и устранение выбросов значений магнитуды округлением до 1 знака пз.'''
my_cat = pd.read_csv('./catalogues/DOUBLE_PREC_DD (2).csv', sep = ';') 
my_cat = my_cat.loc[my_cat.YEAR < 1985]
#my_cat = my_cat.loc[my_cat.latitude > 38.7751667]
#my_cat = my_cat.loc[my_cat.latitude < 38.8081667]
#my_cat = my_cat.loc[my_cat.longitude > -122.778]
#my_cat = my_cat.loc[my_cat.longitude < -122.733]
my_cat = my_cat.reset_index(drop = True)
my_cat.mag = round(my_cat.mag, 1)  
my_cat.SECOND = my_cat.SECOND.astype('int')
my_cat = my_cat.loc[my_cat['mag'] > 1.2]
my_cat = my_cat.reset_index(drop = True)
global N_tot 
N_tot = my_cat.shape
my_cat

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,latitude,longitude,depth,EH1,EH2,AZ,EV,mag,ID
0,1984,1,1,1,19,11,36.08781,-120.22869,10.897,0.020,0.010,96,0.022,1.8,1109386
1,1984,1,1,1,59,27,36.87000,-120.90889,1.592,0.110,0.016,66,0.727,1.5,1109389
2,1984,1,1,3,8,58,40.57227,-124.55933,20.407,1.009,0.083,103,0.080,2.0,1109392
3,1984,1,1,5,1,53,36.24686,-120.39385,6.836,0.059,0.016,106,0.040,1.4,357
4,1984,1,1,7,8,31,36.49920,-121.07855,13.020,0.042,0.020,100,0.102,1.4,1109397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10112,1984,12,31,18,37,5,37.57028,-118.82953,4.763,0.015,0.005,12,0.013,1.7,37589
10113,1984,12,31,19,45,33,37.46175,-118.66582,7.555,0.013,0.009,6,0.014,1.3,37593
10114,1984,12,31,20,43,57,37.46814,-118.57846,10.613,0.011,0.010,173,0.019,2.2,36046
10115,1984,12,31,23,23,8,37.42794,-118.61264,12.437,0.024,0.014,11,0.030,1.6,1124943


In [6]:
n_ = np.load('n_.npy')
resultate = pd.read_csv('./results/nearest_dist_n_index.csv')
''''''
index_o_cn = resultate.Index_o_CN
closest_neigh = resultate.Closest_neigh
less_than_temp = []
for i in range(N_tot[0]-2,-1,-1):
    if closest_neigh[i] < 2.1:
        less_than_temp.append(i)

In [7]:
my_cat = my_cat.drop(my_cat.index[less_than_temp]) 
my_cat = my_cat.reset_index(drop = True)

In [8]:
TIME_LIST = my_cat['YEAR'].astype(str)+ " " + my_cat['MONTH'].astype(str)+" " + my_cat['DAY'].astype(str)+ " " + my_cat['HOUR'].astype(str)+ " " + my_cat['MINUTE'].astype(str)+ " " + my_cat['SECOND'].astype(str)
random.shuffle(TIME_LIST)
my_cat['TIME'] = TIME_LIST
my_cat[['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND']] = my_cat.TIME.str.split(expand=True)
my_cat = my_cat.sort_values(by=['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND'])
my_cat = my_cat.reset_index(drop = True)
N_tot = my_cat.shape

In [9]:
global Pi2, RAD, FLAT
Pi2 = math.pi/2 # тупо пи-на-два
RAD = 0.01745329 # DGR -> RAD
FLAT = 0.993231 # поправка на эллипсоидноть

In [10]:
def eq_dist (fi1, la1, fi2, la2, d1, d2): # fi - широта (latitude), la - долгота (longtitude)
    '''Функция позволяет вычислять расстояение между двумя точками земли с учетом сплюсности и глубины'''    
    #if ((fi1-fi2)==0 and (la1-la2)==0): #проверка на расстояние между одним и тем же землетрясением
    #    return (0.0)  
    col1 = Pi2 - math.atan(FLAT * math.tan(fi1 * RAD))
    col2 = Pi2 - math.atan(FLAT * math.tan(fi2 * RAD))
    
    dlon =  la2 * RAD - la1 * RAD
    
    cosdel = math.sin(col1) * math.sin (col2) * math.cos(dlon) + math.cos(col1) * math.cos(col2)
    delr = math.acos(cosdel)
    colat = Pi2 - (fi1 * RAD + fi2 * RAD) / 2.0
    distan_eq =  math.sqrt((delr * 6371.227 * (1.0 + 0.00337853 * (1./3. -((math.cos(colat))*math.cos(colat)))))**2 + (d2-d1)**2)
    if distan_eq == 0:
        distan_eq = 0.01
    return distan_eq

In [11]:
latitude = my_cat.latitude.to_numpy()
longitude = my_cat.longitude.to_numpy()
depth = my_cat.depth.to_numpy()

In [12]:
array_of_dist = [[0] * N_tot[0] for i in range(N_tot[0])]

for i in tqdm(range((N_tot[0]))):
    for j in range(i+1, N_tot[0]):
        #array_of_dist.append(eq_dist(latitude[k], longitude[k], latitude[j], longitude[j], depth[k], depth[j]))
        array_of_dist[i][j] = eq_dist(latitude[i], longitude[i], latitude[j], longitude[j], depth[i], depth[j])
        #array_of_dist[j][i] = array_of_dist[i][j]

100%|██████████████████████████████████████| 7183/7183 [03:15<00:00, 36.74it/s]


# FFSFFFF

In [13]:
b_value = 0.653687015311375
d_f = 1.6344783067403161

In [14]:
T_ = [[0] * N_tot[0] for i in range(N_tot[0])]

In [15]:
n_ = [[float('inf')] * N_tot[0] for i in range(N_tot[0])]

In [16]:
import datetime
year = my_cat.YEAR.to_numpy().astype(int)
month = my_cat.MONTH.to_numpy().astype(int)
day = my_cat.DAY.to_numpy().astype(int)
hour = my_cat.HOUR.to_numpy().astype(int)
minute = my_cat.MINUTE.to_numpy().astype(int)
second = my_cat.SECOND.to_numpy().astype(int)

In [17]:
@jit
def time_in_seconds (i, j):
    t_i = datetime.datetime(year[i], month[i], day[i], hour[i], minute[i], second[i])
    t_j = datetime.datetime(year[j], month[j], day[j], hour[j], minute[j], second[j])
    t_ij = (t_i - t_j).total_seconds()
    if t_ij == 0:
        return -0.1
    return -t_ij

In [18]:
for i in tqdm(range(N_tot[0]-1,-1,-1)):
    for j in range(i,-1,-1):
        T_[i][j] = -time_in_seconds(i,j)
        T_[j][i] = T_[i][j]

  0%|                                                 | 0/7183 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_9320\1257614798.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "time_in_seconds" failed type inference due to: Unknown attribute 'datetime' of type Module(<module 'datetime' from 'C:\\Users\\admin\\anaconda3\\lib\\datetime.py'>)

File "..\..\..\AppData\Local\Temp\ipykernel_9320\1257614798.py", line 3:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\admin\AppData\Local\Temp\ipykernel_9320\1257614798.py (3)

File "..\..\..\AppData\Local\Temp\ipykernel_9320\1257614798.py", line 3:
<source missing, REPL/exec in use?>

  @jit
C:\Users\admin\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "time_in_seconds" was compiled in object mode without forceobj=True.

File "..\..\..\AppData\Local\Temp\ipykernel_9320\1257614798.py", line 1:
<source mis

In [19]:
for i in tqdm(range(N_tot[0])):
    for j in range(i,N_tot[0]): #ВНИМАНИЕ - ХЗ ПОЧЕМУ ОТ И БЕРУ. ХОЧУ ПРОВЕРИТЬ
        if i == j:
            continue
        n_[j][i] = abs(T_[j][i])*(array_of_dist[i][j]**d_f)*(10**((-b_value)*my_cat.mag[i]))
        #n_[j][i] = n_[i][j]

100%|██████████████████████████████████████| 7183/7183 [08:10<00:00, 14.63it/s]


In [20]:
n_[20]

[259941450.48923767,
 147021445.98616555,
 935727732.5628778,
 628277934.268251,
 688205634.5420054,
 1994720.0470848016,
 22064606.096710585,
 76339641.12256293,
 685406.5340888766,
 1303871.9415111241,
 69633225.08259112,
 1071006251.7291534,
 30011739.932340242,
 20743009.090193123,
 38379574.85137822,
 45163640.884009525,
 25238207.85592545,
 8785579.041031344,
 17179081.11717117,
 1751549.2904112188,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 in

In [21]:
N = [0] * N_tot[0]
D = [0] * N_tot[0]
for i in range(len(n_)):
    D[i] = round(math.log10((min(n_[i][:]))),1)
    N[i] = n_[:][i].index(min(n_[i][:]))
resultate = pd.DataFrame({'Closest_neigh':D, 'Index_o_CN':N})

In [22]:
Dist_to_Nearest = resultate.Closest_neigh.value_counts(normalize=True)
from pathlib import Path  
filepath = Path('./results/nearest_dist_rand.csv')
Dist_to_Nearest.to_csv(filepath)